# Getting to your Data in Azure Notebooks
Jupyter provides the basis of the Azure Notebooks user experience. But it doesn't provide us any data. This notebook provides samples of how you might retrieve data to use from within your own notebooks.

There are many ways to get your data in your notebooks ranging from using curl or leveraging the [Azure](http://pypi.python.org/pypi/azure) package to access a variety of data all while working from a Jupyter Notebook. See the table of contents below to jump to a particular example.

## Table of Contents
- [Use curl to retrieve a file from GitHub](#curl)
- [Interacting with Azure Blobs](#blobs)
- [Using Azure Table Storage](#tablestorage)
- [Providing Read Only Access to Azure Storage through Shared Access Signatures](#sharedaccess)
- [Cleaning up created blobs and tables](#cleanup)
- [Using SQL](#usingsql)
- [Other Resources](#otherways)

## Use `curl` to retrieve a file from GitHub <a name="curl"></a> 

We can call bash commands by starting our line with a `!`. In this way we can just curl a file down from the internet, like this csv about oil prices.

In [1]:
!curl -L https://raw.githubusercontent.com/petroleum101/figures/db46e7f48b8aab67a0dfe31696f6071fb7a84f1e/oil_price/oil_price.csv -o oil_price.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5190  100  5190    0     0  30005      0 --:--:-- --:--:-- --:--:-- 30174


Then, if we wanted to do something with it, we might choose to load it into pandas.

In [2]:
import pandas
dataframe = pandas.read_csv('oil_price.csv')
dataframe.head()

Date  BP us dollar of the day  \
0  1861                     0.49   
1  1862                     1.05   
2  1863                     3.15   
3  1864                     8.06   
4  1865                     6.59   

   US Domestic Oil Price (US $) PET_F000000__3_A  \
0                                           0.49   
1                                           1.05   
2                                           3.15   
3                                           8.06   
4                                           6.59   

   U.S. Landed Costs of Saudi Arabian Light Crude Oil isa4990008a  \
0                                                NaN                
1                                                NaN                
2                                                NaN                
3                                                NaN                
4                                                NaN                

   BP Arabian Light posted at Ras Tanura.  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   

   Crude Oil Prices: Brent ÃÂÃÂ¢ÃÂÃÂÃÂÃÂ Europe  ACOILBRENTEU  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   Crude Oil Prices: West Texas Intermediate (WTI)  ACOILWTICO  \
0                                                NaN             
1                                                NaN             
2                                                NaN             
3                                                NaN             
4                                                NaN             

    Imported Crude Oil Price (refiner average imported crude oil acquisition cost) (PET.R1300____3.A)  \
0                                                NaN                                                    
1                                                NaN                                                    
2                                                NaN                                                    
3                                                NaN                                                    
4                                                NaN                                                    

   U.S. Crude Oil Domestic Acquisition Cost by Refiners, Annual (PET.R1200____3.A)  \
0                                                NaN                                 
1                                                NaN                                 
2                                                NaN                                 
3                                                NaN                                 
4                                                NaN                                 

   Dubai Crude Oil Price opendataforafrica.org/IMFPCP2014Jan  
0                                                NaN          
1                                                NaN          
2                                                NaN          
3                                                NaN          
4                                                NaN

## Interacting with Azure Blobs <a name="blobs"></a>

We can also use Azure Storage to store our data. It also makes it pretty straightforward to keep our data private or public. The below code shows using private keys first. Then, in the [shared access](#sharedaccess) section a shared access signature for read-only access is created.


Before we can do anything though, we need an Azure Storage Account. Read the [documentation](https://azure.microsoft.com/en-us/documentation/articles/storage-create-storage-account/#create-a-storage-account) article on creating storage accounts or [create a storage account using the Azure SDK](http://azure-sdk-for-python.readthedocs.io/en/latest/resourcemanagementstorage.html#create-storage-account).

You can put content into blobs using [AzCopy](https://azure.microsoft.com/en-us/documentation/articles/storage-use-azcopy/) or by using the Python Azure SDK as shown in the example below.

Once you retrieve your account and key, you can enter them below.
This code will create a container and blob in the azure storage you provide. Then we will read that blob back.

In [3]:
azure_storage_account_name = None
azure_storage_account_key = None

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [4]:
!pip install azure-storage==0.32.0

You are using pip version 8.1.1, however version 8.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
from azure.storage.blob import BlockBlobService

# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

# Once we are in the account we can create a container
blob_service.create_container('azure-notebooks-data')

# Insider a container we can create other containers or a blob
blob_service.create_blob_from_text('azure-notebooks-data', 'sample.txt', 'your text file content would go here')

# There are also methods to list containers and blobs
containers = blob_service.list_containers()
blobs = blob_service.list_blobs('azure-notebooks-data')

# We can also read our blob from azure and get the text.
blob_service.get_blob_to_path('azure-notebooks-data', 'sample.txt', 'sample.txt')

!cat sample.txt

your text file content would go here

## Using Azure Table Storage<a name="tablestorage"></a>

Azure Table Storage can be used in much the same way as Blob Storage. Below you will find creating a table in a storage account, adding rows, removing rows, and querying for data.

In [7]:
from azure.storage.table import TableService
import IPython

# We start by connecting to our table
table_service = TableService(azure_storage_account_name, azure_storage_account_key)

# Creating a table in Azure Storage is straightforward
table_name = 'azurenotebookstesttable'
table_service.create_table(table_name)

# You can insert entities to the table
entity = {'PartitionKey': 'testItems', 'RowKey': '0', 'age':1}
table_service.insert_entity(table_name, entity)
table_service.insert_entity(table_name, {'PartitionKey': 'testItems', 'RowKey': '10', 'age':2, 'eyecolor':'blue'})

# To query for entities you can use the following code
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after inserts ===')
IPython.display.display_pretty([i for i in queried_entities])

# You can delete an entity by using its partition and row key.

table_service.delete_entity(table_name, 'testItems', '0')
                                         
# We can query to show we have removed the entity
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after delete ===')
IPython.display.display_pretty([i for i in queried_entities])

=== Queried rows after inserts ===


[{'PartitionKey': 'testItems',
  'RowKey': '0',
  'Timestamp': datetime.datetime(2016, 5, 20, 17, 26, 9, 531793, tzinfo=tzlocal()),
  'age': 1,
  'etag': 'W/"datetime\'2016-05-20T17%3A26%3A09.5317932Z\'"'},
 {'PartitionKey': 'testItems',
  'RowKey': '10',
  'Timestamp': datetime.datetime(2016, 5, 20, 17, 26, 9, 569828, tzinfo=tzlocal()),
  'age': 2,
  'etag': 'W/"datetime\'2016-05-20T17%3A26%3A09.5698289Z\'"',
  'eyecolor': 'blue'}]

=== Queried rows after delete ===


[{'PartitionKey': 'testItems',
  'RowKey': '10',
  'Timestamp': datetime.datetime(2016, 5, 20, 17, 26, 9, 569828, tzinfo=tzlocal()),
  'age': 2,
  'etag': 'W/"datetime\'2016-05-20T17%3A26%3A09.5698289Z\'"',
  'eyecolor': 'blue'}]

## Providing Read Only Access to Azure Storage through Shared Access Signatures <a name="sharedaccess"></a>

Sometimes you want to share your data but you don't want to give them the ability to edit the dataset. Shared Access Signatures allow you to share your data and provide whatever level of control you want to the receiver. A common use case is to provide read only access to a user so they can read your data but not edit it.

Below, we create a shared access signature for our table (this also works with blobs) with read permissions. We show that we can read the table but we show that we can't write. With tables you also need to provide permission to query.

### Creating a Shared Access Signature

In [8]:
from azure.storage.blob.models import BlobPermissions
from datetime import datetime, timedelta

sas_token = blob_service.generate_blob_shared_access_signature(
    'azure-notebooks-data',
    'sample.txt',
    BlobPermissions.READ,
    datetime.utcnow() + timedelta(hours=1)
)

sas_token

'se=2016-05-20T18%3A25%3A13Z&sig=rskxaKrEtnWcvVzfjW2rdofv5gWV9NVLgixH6HbkrK4%3D&sp=r&sv=2015-07-08&sr=b'

### Using a Shared Access Signature

In [9]:
# Create a service and use the SAS 
sas_blob_service = BlockBlobService( 
    account_name=azure_storage_account_name, 
    sas_token=sas_token,
)

sas_blob_service.get_blob_to_text('azure-notebooks-data', 'sample.txt').content

'your text file content would go here'

## Cleaning up our blobs and tables <a name="cleanup"></a>

In [10]:
# Finally, let's clean up the resources created.
# Delete the blob, container, and table we created
blob_service.delete_blob('azure-notebooks-data', 'sample.txt')
blob_service.delete_container('azure-notebooks-data')
table_service.delete_table('azurenotebookstesttable')

True

## Using SQL  <a name="usingsql"></a>
With the assistance of the pyodbc library we can access our SQL Servers in Microsoft Azure. To create a SQL Server you can see the documentation for [Creating and Using Azure SQL Documentation](https://azure.microsoft.com/en-us/documentation/articles/sql-database-develop-python-simple/).


In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc

server = 'yourserver.database.windows.net'
database = 'yourdatabase'
username = 'yourusername'
password = 'yourpassword'

driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("select @@VERSION")
row = cursor.fetchone()
if row:
    print row

In [ ]:
#PYMSSQL --> NOTE the connection parameter settings for pymssql are different from pyodbc above. 
#pymssql.connect("xxx.database.windows.net", "username@xxx", "password", "db_name")
#see details : http://pymssql.org/en/latest/azure.html

import pymssql
conn = pymssql.connect("yourserver.database.windows.net", "yourusername@yourserver", "yourpassword", "yourdatabase")
cursor2 = conn.cursor()
cursor2.execute("select @@VERSION")
row = cursor2.fetchone()
if row:
    print( row)

## Other Resources <a name="otherways"></a>

This notebook does not show all possible ways you can access your data. Below are a few other examples of how you may access data. 

### Links

[Azure Data Factory](https://azure.microsoft.com/en-us/services/data-factory/)

[Copy Wizard for Azure Data Factory](https://azure.microsoft.com/en-us/updates/code-free-copy-wizard-for-azure-data-factory/)

